[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/notebooks/blob/main/camenduru's_waifu_diffusion.ipynb)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!pip install -qq git+https://github.com/camenduru/diffusers.git
!pip install -qq transformers ftfy datasets

import torch, os, gc
from diffusers import StableDiffusionPipeline
from torch import autocast

pipe = StableDiffusionPipeline.from_pretrained("hakurei/waifu-diffusion", revision="fp16", torch_dtype=torch.float16).to("cuda")
pipe.safety_checker = lambda images, **kwargs: [images, [False] * len(images)]

batch_idx = 0
outputs_path = "/content/gdrive/MyDrive/AI/WaifuDiffusion"

In [ ]:
from PIL.PngImagePlugin import PngInfo
metadata = PngInfo()

is_tile = False #@param {type:'boolean'}
if(is_tile):
	def patch_conv(cls):
		init = cls.__init__
		def __init__(self, *args, **kwargs):
			return init(self, *args, **kwargs, padding_mode='circular')
		cls.__init__ = __init__
	patch_conv(torch.nn.Conv2d)
 
images_path_name = "000" #@param {type:'string'}
num_batch_images = 1000 
height = 512 #@param {type:'integer'}
width = 512 #@param {type:'integer'}

path = os.path.join(outputs_path, images_path_name)

if os.path.exists(path) == False:
  os.mkdir(path)

prompt = 'manga' #@param {type:'string'}
metadata.add_text("Prompt", f"{prompt}")
by = 'camenduru' #@param {type:'string'}
metadata.add_text("by", f"{by}")

for i in range(num_batch_images):
    gc.collect()
    torch.cuda.empty_cache()
    with autocast("cuda"):
      image = pipe(prompt, height=height, width=width, num_inference_steps=50, eta=0.0, guidance_scale=7.5)["sample"][0]

    image.save(f"{path}/{images_path_name}_{batch_idx}.png", pnginfo=metadata)
    batch_idx += 1